# Functional Programming 

Functional programming in Python:

- map,
- lazy computation,
- lambda functions,
- filter,
- reduce.

Three things:

1. no side effects,
2. variables don't vary / immutable data,
3. first class functions - no objects and no state,


## Resources

[Functional Programming in Python
By Marcus Sanatan](https://stackabuse.com/functional-programming-in-python/)

[Clean Architecture - Uncle Bob Martin](https://www.amazon.co.uk/Clean-Architecture-Craftsmans-Software-Structure/dp/0134494164)

[What is Functional Programming? - Scott Murphy](https://www.youtube.com/watch?v=KHojnWHemO0)

## What is functional programming?

Three things:

1. no side effects,
2. variables don't vary / immutable data,
3. first class functions - no objects and no state,

Examples of functional programming languages include Lisp, Haskell, Erlang, Clojure.

Also a useful idea when thinking about implementations

[John Carmack (on Parallel Implementations)](http://sevangelatos.com/john-carmack-on-parallel-implementations/) notes it is eaiser to switch out implementations if ideas are expressed as pure functions.  Internal state & multiple entry points make programming harder.

## 1. No side effects

- same inputs -> same outputs - always,
- idempotent - things always run / perform the same way,
- no dependency on the state of the outside world.

Not doing something like:

In [ ]:
def pipeline(data):
    new_data = clean(data)
    database.save(new_data)
    database.load(features)
    return features

## 2. Variables that don't vary / Immutable data

Variables are only initialized:

- they are never changed,
- this avoids problems such as race conditions / deadlocks.

Variables being immutable means we can't do:

In [ ]:
#  can't do this - mutates the object in place
def f(x):
    x += 1
    return x

#  can do this - creates a new object
def f(x):
    x = x + 1
    return x

## 3. Functions are first class

First class citizens means we can pass functions around like other variables (also known as higher order functions).

Below we pass the `sum` function into a generic `reducer` function (there is no Python built-in for reduce):

In [ ]:
def reducer(func, data):
    return func(data)

data = [1, 2, 3]
reducer(sum, data)

Passing in another function (`len`) gives different results - the length of our data:

In [ ]:
reducer(len, data)

## Map

Applying a function to each element of an iterable - similar to `df.apply` in pandas:

In [ ]:
def lower(s):
    return s.lower()

cities = ['Berlin', 'Auckland', 'London', 'Sheffield']
m = map(lower, cities)
m

We can see that Python has returned a map object, not the transformed data.  This is an example of **lazy computation**, which is a two step process:
1. build a pipeline/graph

2. put data through it when needed

Examples include Tensorflow 1, Spark, Python generators.

We can get the next element as we would for any kind of generator:

As we are more impatient than lazy, we can get all the processed data by calling `list` on the generator:

In [ ]:
list(map(lower, cities))

In [ ]:
cities

## Lambda functions

Anonymous = the function is not assigned to a variable. In Python we can have objects with no variable reference (until they get garbage collected :)

We can do the same example as above using a lambda function (which is anonmyous):

In [ ]:
list(map(lambda x: x.lower(), cities))

The object we define above is a lambda function:

In [ ]:
lw = lambda x: x.lower()

We can do more complex things in lambdas, such as accessing elements of the input data:

In [ ]:
populations = [
    ('Berlin', 3.7, 'eu'),
    ('Auckland', 1.7, 'pac'),
    ('London', 8.9, 'eu'),
    ('Sheffield', 0.5, 'eu')
]

list(map(lambda x: (x[0], x[1] * 1000), populations))

We have total flexibility in **what data structure** we use in the iterable, and **how we interact with it** in the lambda.  We have decomposed and separated the data from the behaviour!

We could map over a sequence of namedtuples, and access elements using the attribute `.` syntax:

In [ ]:
from collections import namedtuple

pop = namedtuple('city', ['city', 'population', 'continent'])

populations = [pop(*p) for p in populations]

list(map(lambda x: (x.city, x.population * 1000), populations))

## Reduce

Reduce will aggregate a sequence to single values (either a single value for the entire sequence, or one single value per group).  Also known as aggregation or a groupby.

Above we defined a function `reducer` to demonstrate that functions are first class citizens in Python.  

The Python standard library also has it's own reduce function - `functools.reduce`.  This function operates on each element and accumulates, returning a single aggregated example.

We can use this `reduce` function on our `populations` dataset:

- define a lambda function that adds on the population of the sequence to the total,
- our sequence populations,
- an initial value of `0`.

This is how we would do this in normal Python - iterate over the list and increment a float:

In [ ]:
total = 0
for p in populations:
    total += p.population
total

The equivilant using `functools.reduce`:

In [ ]:
from functools import reduce

reduce(lambda total, pop: total + pop[1], populations, 0)

We can also use this `reduce` function to groupby continent:

In [ ]:
reduce?

In [ ]:
def gb(acc, city):
    acc[city.continent].append(city.city)
    return acc

reduce(
    gb,
    populations,
    {'eu': [], 'pac': []}
)

## Filter

Tests each element, keeps those that pass - similar to boolean masking in pandas/numpy:

In [ ]:
list(filter(lambda x: x[1] > 1.0, populations))

## Practical

Create a data processing pipeline that selects the cities that have populations greater than the average of all cities

Two steps:
1. `reduce` to find the average of all cities,
2. `filter` to select using cities above that average.

This can be done in a single reduce step - suggest first getting the two step solution working :)

## Practical

Implement the same pipeline using two list comprehensions

## Question

In Python, any combination of `map` & `filter` can be done with a single list comprehension - why do we need to use two in the example above?

## Practical

Create a data processing pipeline that finds the average population for both continents:
1. reduce to (key, (populations)),
2. map from (key, (populations)) to (key, avg).